In [1]:
from taxcalc import *
from bokeh.io import show, output_notebook
from bokeh.layouts import column
from bokeh.plotting import figure
from bokeh.models import HoverTool, ColumnDataSource, NumeralTickFormatter
from collections import OrderedDict
import copy
import pandas as pd
from notebookfunctions import distribution, index_list, percentile
output_notebook()

Loading BokehJS ...

In [2]:
# Data from IRS-SOI Tax Stats. Used for comparison
soi_stats = pd.read_csv('soi_stats.csv', index_col=0)  # Equivalent to tax-calc diagnostic table
soi_income = pd.read_csv('soi_income_stats.csv')  # Distribution of income items
soi_deductions = pd.read_csv('soi_deductions.csv', index_col=0)  # Itemized deductions
soi_deductions['index'] = soi_deductions.index

In [3]:
# Read in new PUF and associated data
gf = pd.read_csv('../taxdata/stage1/growfactors.csv')
wt = pd.read_csv('../taxdata/puf_stage2/puf_weights.csv')
puf = pd.read_csv('../taxdata/puf_data/puf.csv')
adj = pd.read_csv('../taxdata/puf_stage3/puf_ratios.csv', index_col=0)
adj = adj.transpose()

In [4]:
# base calculator
# temporary growth rates
grow_temp = Growfactors('taxcalc/growfactors_temp.csv')
recs = Records(gfactors=grow_temp)
pols = Policy()
calc = Calculator(records=recs, policy=pols)
calc.advance_to_year(2014)
calc.calc_all()

You loaded data for 2009.
Your data include the following unused variables that will be ignored:
  n1821
Tax-Calculator startup automatically extrapolated your data to 2013.


In [5]:
# new calculator
growf = Growfactors('../taxdata/stage1/growfactors.csv')
rec = Records(puf, gfactors=growf,
              weights=wt, adjust_ratios=adj,
              start_year=2011)
pol = Policy(gfactors=growf)
consump = Consumption(start_year=2011, num_years=17)
behave = Behavior(start_year=2011, num_years=17)
#params = ParametersBase(start_year=2011, num_years=17)
calc_new = Calculator(records=rec, policy=pol, consumption=consump, behavior=behave)
calc_new.advance_to_year(2014)
print(calc_new.policy_current_year())
calc_new.calc_all()

You loaded data for 2011.
Tax-Calculator startup automatically extrapolated your data to 2013.
2014


In [6]:
# list of variables used throughout notebook
var_list = ['e00200', 'e00300', 'e00600', 'e00650', 'e00900', 'c00100',
            's006', 'e17500', 'c04470', 'e18400', 'e18500', 'e19200', 'e19800', 'e20100',
            'e20400', 'g20500', 'iitax', 'payrolltax', 'combined']
# create data frames for both calculators
calc_df = calc.dataframe(var_list)
new_df = calc_new.dataframe(var_list)

# Distribution Tables

In [7]:
curr_table, _ = calc.distribution_tables(None)
print ('Current PUF')
curr_table

ValueError: Buffer dtype mismatch, expected 'Python object' but got 'long'

Exception ignored in: 'pandas._libs.lib.is_bool_array'
ValueError: Buffer dtype mismatch, expected 'Python object' but got 'long'


Current PUF


,s006,c00100,num_returns_StandardDed,standard,num_returns_ItemDed,c04470,c04600,c04800,taxbc,c62100,...,c09600,c05800,c07100,othertaxes,refund,iitax,payrolltax,combined,expanded_income,aftertax_income
0,"16,221,793","-87,307,405,375","11,146,637","96,407,359,841","6,087","264,420,507","66,734,350,903","837,996,365","80,929,000","-93,697,347,981",...,0,"80,929,000","6,971,601","81,400,811","2,526,565,011","-2,371,206,801","5,813,976,106","3,442,769,306","-83,056,124,135","-86,498,893,441"
1,"16,222,840","97,696,000,447","13,632,755","114,464,028,623","54,252","427,192,044","79,940,587,312","4,098,599,114","366,554,922","97,335,286,959",...,"7,933,783","374,488,705","7,163,854","67,319,871","12,374,838,446","-11,940,193,724","13,166,598,731","1,226,405,007","136,744,925,045","135,518,520,038"
2,"16,221,490","163,468,877,532","13,230,158","123,397,911,082","291,268","3,127,562,614","105,324,141,284","23,615,280,936","2,317,284,610","161,287,246,966",...,"28,785,086","2,346,069,695","313,479,968","95,718,940","26,344,313,834","-24,216,005,166","22,653,526,956","-1,562,478,210","243,042,824,393","244,605,302,603"
3,"16,223,700","244,271,226,308","14,193,383","126,510,866,481","665,097","8,675,779,270","110,406,827,660","64,167,133,999","6,728,920,955","238,716,968,823",...,"29,752,265","6,758,673,220","1,317,901,363","190,009,074","23,461,455,686","-17,830,674,754","32,516,776,026","14,686,101,272","355,123,480,068","340,437,378,795"
4,"16,222,475","362,531,321,371","14,165,525","128,715,334,378","1,497,871","20,700,893,573","116,829,492,149","139,146,548,563","16,078,090,151","349,707,249,795",...,"44,135,661","16,122,225,812","3,488,450,770","262,896,068","17,584,563,210","-4,687,892,100","46,562,008,652","41,874,116,551","491,474,693,643","449,600,577,091"
5,"16,221,683","524,486,317,326","13,178,967","124,460,625,287","2,882,627","43,092,458,811","118,837,408,328","265,330,113,463","32,309,372,364","498,816,566,335",...,"40,562,634","32,349,934,997","5,828,747,564","426,620,413","8,995,351,586","17,952,456,261","63,669,887,689","81,622,343,950","660,912,932,518","579,290,588,568"
6,"16,222,444","748,546,418,528","11,388,577","115,287,352,674","4,803,785","80,625,803,708","125,281,678,668","437,438,543,361","57,283,794,507","700,594,959,360",...,"38,919,584","57,322,714,090","8,154,522,127","680,661,675","2,753,987,998","47,094,865,639","85,917,017,696","133,011,883,336","882,702,922,566","749,691,039,230"
7,"16,222,827","1,092,349,731,112","9,043,469","98,828,370,778","7,172,046","132,250,968,817","139,442,917,664","723,997,183,246","104,415,929,974","1,018,381,692,957",...,"60,125,018","104,476,054,992","11,072,525,677","1,012,981,743","1,298,281,698","93,118,229,361","124,635,761,651","217,753,991,012","1,204,302,332,176","986,548,341,164"
8,"16,221,896","1,655,218,919,934","5,830,283","69,495,223,098","10,390,784","223,921,661,490","164,449,691,635","1,198,406,293,993","187,012,834,871","1,535,575,163,967",...,"293,968,886","187,306,803,758","14,524,350,223","1,615,376,719","1,548,159,725","172,849,670,528","201,791,432,199","374,641,102,727","1,780,169,573,807","1,405,528,471,079"
9,"16,223,470","5,010,404,019,291","1,996,496","24,063,641,503","14,214,627","596,219,211,775","156,981,070,943","4,239,681,834,066","1,061,876,168,463","4,769,377,025,799",...,"38,841,090,217","1,100,717,258,680","29,120,331,277","30,368,805,793","923,418,383","1,101,042,314,813","357,073,176,859","1,458,115,491,672","5,288,418,667,894","3,830,303,176,222"


In [8]:
new_table, _ = calc_new.distribution_tables(None)
print ('New PUF')
new_table

ValueError: Buffer dtype mismatch, expected 'Python object' but got 'long'

Exception ignored in: 'pandas._libs.lib.is_bool_array'
ValueError: Buffer dtype mismatch, expected 'Python object' but got 'long'


New PUF


,s006,c00100,num_returns_StandardDed,standard,num_returns_ItemDed,c04470,c04600,c04800,taxbc,c62100,...,c09600,c05800,c07100,othertaxes,refund,iitax,payrolltax,combined,expanded_income,aftertax_income
0,"16,206,016","-43,955,548,157","11,504,053","98,088,334,344","1,536","204,936,571","64,220,092,639","957,908,230","70,501,127","-44,072,448,419",...,0,"70,501,127","3,036,696","94,705,288","1,780,987,150","-1,618,817,431","5,795,414,443","4,176,597,012","-34,255,616,751","-38,432,213,763"
1,"16,205,646","110,475,248,450","13,947,869","118,430,150,466","75,133","783,505,875","82,105,322,994","6,481,522,962","584,233,154","109,915,759,517",...,"45,547,935","629,781,090","7,860,622","61,020,742","14,240,643,167","-13,557,701,957","15,184,895,783","1,627,193,826","151,056,018,367","149,428,824,540"
2,"16,207,848","177,938,900,865","13,961,991","126,677,851,196","304,623","3,508,811,987","105,651,296,378","29,588,592,647","2,873,041,480","175,694,229,341",...,"109,405,262","2,982,446,742","171,877,362","103,523,914","26,935,974,560","-24,021,881,267","24,819,818,644","797,937,377","253,844,779,188","253,046,841,811"
3,"16,206,368","260,979,946,631","14,478,763","131,996,868,587","617,871","7,836,595,201","112,539,923,210","73,509,150,193","7,855,855,388","256,464,494,957",...,"138,732,701","7,994,588,089","1,065,931,612","190,203,222","23,177,106,699","-16,058,247,001","35,396,393,499","19,338,146,498","370,136,285,209","350,798,138,711"
4,"16,206,466","383,411,145,296","14,311,291","130,347,117,183","1,515,505","20,714,303,000","116,822,779,914","154,986,593,621","18,026,266,009","371,100,962,564",...,"164,605,136","18,190,871,145","2,962,911,887","275,321,807","16,021,423,625","-518,142,560","49,667,788,877","49,149,646,316","507,531,873,614","458,382,227,298"
5,"16,206,726","543,216,446,556","13,201,114","126,587,818,026","2,894,250","41,830,188,740","120,531,350,879","278,561,088,867","33,456,313,515","519,174,968,180",...,"212,795,449","33,669,108,964","4,874,538,753","386,082,573","7,421,200,937","21,759,451,847","66,207,518,579","87,966,970,426","678,569,616,339","590,602,645,914"
6,"16,206,360","769,481,535,126","11,507,686","117,236,831,940","4,672,148","75,243,179,607","125,863,642,008","458,964,673,984","59,937,635,743","725,880,107,489",...,"205,536,546","60,143,172,289","6,643,497,223","535,854,229","1,564,416,792","52,471,112,503","89,529,425,057","142,000,537,560","899,033,672,667","757,033,135,107"
7,"16,206,802","1,113,299,775,875","9,043,900","99,788,803,825","7,157,976","130,517,764,665","140,406,594,535","744,808,414,336","107,292,201,357","1,040,579,924,455",...,"265,126,208","107,557,327,565","8,539,132,004","947,011,218","148,697,161","99,816,509,618","126,599,797,935","226,416,307,554","1,224,236,532,027","997,820,224,474"
8,"16,206,184","1,652,835,233,581","5,906,677","70,037,638,314","10,296,516","219,753,483,111","163,404,215,869","1,202,833,971,215","186,962,072,271","1,535,596,279,875",...,"390,356,906","187,352,429,177","10,662,017,410","1,376,516,076","25,210,223","178,041,717,619","198,130,419,337","376,172,136,956","1,779,605,018,337","1,403,432,881,381"
9,"16,207,154","4,964,347,167,184","2,042,789","25,010,545,616","14,153,796","585,088,760,983","155,468,866,391","4,205,097,802,306","1,041,518,623,845","4,748,915,325,142",...,"40,880,623,983","1,082,399,247,828","22,514,477,283","29,747,560,238","1,797,805","1,089,630,532,979","345,673,607,519","1,435,304,140,498","5,276,256,849,959","3,840,952,709,461"


# Diagnostic Table Comparison

In [9]:
curr_diag = calc.diagnostic_table(1)
curr_diag

,2014
Returns (#m),162.2
AGI ($b),"9,811.7"
Itemizers (#m),42.0
Itemized Deduction ($b),"1,109.3"
Standard Deduction Filers (#m),107.8
Standard Deduction ($b),925.9
Personal Exemption ($b),"1,115.8"
Taxable Income ($b),"7,096.7"
Regular Tax ($b),"1,468.5"
AMT Income ($b),"9,276.1"


In [10]:
new_diag = calc_new.diagnostic_table(1)
new_diag

,2014
Returns (#m),162.1
AGI ($b),"9,932.0"
Itemizers (#m),41.7
Itemized Deduction ($b),"1,085.5"
Standard Deduction Filers (#m),109.9
Standard Deduction ($b),957.0
Personal Exemption ($b),"1,129.7"
Taxable Income ($b),"7,155.8"
Regular Tax ($b),"1,458.6"
AMT Income ($b),"9,439.2"


In [11]:
diag_data = pd.DataFrame()
diag_data['SOI'] = soi_stats['Value']
diag_data['New'] = new_diag[2014]
diag_data['Current'] = curr_diag[2014]
diag_data['% Change'] = ((new_diag[2014] / curr_diag[2014]) - 1) * 100
diag_data['New - SOI'] = new_diag[2014] - diag_data['SOI']
diag_data['Current - SOI'] = curr_diag[2014] - diag_data['SOI']
diag_data

,SOI,New,Current,% Change,New - SOI,Current - SOI
Returns (#m),148.6,162.1,162.2,-0.1,13.5,13.6
AGI ($b),"9,771.0","9,932.0","9,811.7",1.2,161.0,40.7
Itemizers (#m),44.0,41.7,42.0,-0.7,-2.3,-2.0
Itemized Deduction ($b),"1,206.7","1,085.5","1,109.3",-2.1,-121.2,-97.4
Standard Deduction Filers (#m),117.4,109.9,107.8,1.9,-7.5,-9.6
Standard Deduction ($b),876.2,957.0,925.9,3.4,80.8,49.7
Personal Exemption ($b),"1,121.6","1,129.7","1,115.8",1.3,8.1,-5.8
Taxable Income ($b),"6,997.9","7,155.8","7,096.7",0.8,157.9,98.8
Regular Tax ($b),nan,"1,458.6","1,468.5",-0.7,nan,nan
AMT Income ($b),nan,"9,439.2","9,276.1",1.8,nan,nan


# Extended Diagnostic Table

In [12]:
print ('Current Calculator')
calc.diagnostic_table(14)

Current Calculator


,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024,2025,2026,2027
Returns (#m),162.2,164.4,167.1,170.1,172.9,175.4,177.7,179.9,182.2,184.4,186.7,189.0,191.2,192.5
AGI ($b),"9,811.7","10,477.8","10,751.9","11,266.8","11,809.6","12,224.2","12,663.8","13,144.4","13,675.0","14,239.5","14,831.2","15,449.0","16,033.1","16,719.2"
Itemizers (#m),42.0,43.0,44.0,45.1,20.0,20.1,20.7,21.4,22.0,22.6,23.3,24.0,50.3,51.5
Itemized Deduction ($b),"1,109.3","1,172.3","1,224.8","1,294.1",631.8,648.6,685.1,726.3,767.5,809.8,855.8,903.4,"1,927.3","2,023.8"
Standard Deduction Filers (#m),107.8,108.8,110.2,111.5,139.1,140.8,142.1,143.3,144.6,145.8,147.1,148.3,124.4,124.8
Standard Deduction ($b),925.9,949.9,962.7,981.0,"2,360.7","2,437.0","2,511.1","2,586.2","2,666.2","2,746.6","2,829.0","2,913.8","1,328.9","1,365.3"
Personal Exemption ($b),"1,115.8","1,138.2","1,164.0","1,175.5",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"1,530.4","1,576.6"
Taxable Income ($b),"7,096.7","7,662.0","7,844.7","8,269.4","9,101.5","9,432.5","9,772.9","10,148.8","10,569.3","11,020.8","11,493.7","11,989.4","11,842.9","12,336.5"
Regular Tax ($b),"1,468.5","1,588.0","1,628.1","1,721.1","1,712.3","1,774.5","1,837.9","1,909.5","1,990.6","2,079.1","2,172.1","2,269.4","2,478.4","2,573.1"
AMT Income ($b),"9,276.1","9,909.7","10,157.3","10,634.4","11,353.2","11,755.2","12,167.9","12,618.1","13,118.6","13,651.8","14,209.7","14,792.7","15,087.0","15,713.7"


In [13]:
print ('New Calculator')
calc_new.diagnostic_table(14)

New Calculator


,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024,2025,2026,2027
Returns (#m),162.1,164.3,167.0,170.0,172.8,175.4,177.7,180.0,182.2,184.5,186.8,189.0,191.3,193.6
AGI ($b),"9,932.0","10,600.7","10,875.3","11,397.7","11,954.1","12,384.2","12,834.6","13,326.3","13,866.8","14,442.5","15,045.1","15,674.0","16,257.3","16,944.8"
Itemizers (#m),41.7,42.8,43.9,45.1,19.4,19.5,20.0,20.7,21.3,22.0,22.7,23.4,50.1,50.6
Itemized Deduction ($b),"1,085.5","1,146.7","1,199.2","1,267.3",612.7,627.6,662.9,701.9,742.9,786.7,831.3,879.4,"1,886.7","1,971.4"
Standard Deduction Filers (#m),109.9,110.8,112.2,113.6,141.8,143.7,145.1,146.4,147.7,149.0,150.3,151.6,127.2,128.7
Standard Deduction ($b),957.0,980.5,992.5,"1,010.7","2,422.9","2,505.5","2,582.9","2,662.7","2,745.5","2,828.1","2,915.5","3,003.0","1,378.9","1,426.1"
Personal Exemption ($b),"1,129.7","1,152.6","1,178.4","1,190.1",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"1,551.8","1,597.2"
Taxable Income ($b),"7,155.8","7,727.4","7,914.1","8,345.2","9,172.7","9,514.9","9,862.1","10,244.8","10,670.8","11,128.9","11,609.0","12,111.1","11,984.9","12,511.4"
Regular Tax ($b),"1,458.6","1,576.8","1,619.9","1,713.3","1,704.8","1,768.7","1,832.6","1,904.2","1,985.5","2,074.6","2,167.6","2,265.5","2,476.4","2,590.0"
AMT Income ($b),"9,439.2","10,078.2","10,328.2","10,815.3","11,553.3","11,974.6","12,401.3","12,866.3","13,379.5","13,925.4","14,497.0","15,092.0","15,387.7","16,034.0"


# Income Levels

In [14]:
def pct_diff(current, new):
    return ((new / current) - 1) * 100

In [15]:
inc_dict = OrderedDict()
inc_dict['New'] = []
inc_dict['Current'] = []
inc_dict['SOI'] = []
inc_dict['Pct Diff'] = []
inc_dict['New - SOI'] = []
inc_dict['Current - SOI'] = []
inc_vars = ['e00200', 'e00300', 'e00600', 'e00650', 'e00900']
soi_inc_vars = ['WAS', 'INT', 'ODIV', 'QDIV', 'BIZ']
for tc, soi in zip(inc_vars, soi_inc_vars):
    new_val = calc_new.weighted_total(tc)
    curr_val = calc.weighted_total(tc)
    soi_val = soi_income[soi].sum()
    inc_dict['New'].append(new_val)
    inc_dict['Current'].append(curr_val)
    inc_dict['SOI'].append(soi_val)
    inc_dict['Pct Diff'].append(pct_diff(curr_val, new_val))
    inc_dict['New - SOI'].append(new_val - soi_val)
    inc_dict['Current - SOI'].append(curr_val - soi_val)
# create data frame to display totals better
inc_df = pd.DataFrame.from_dict(inc_dict)
inc_df.index = ['WAS', 'Taxable Interest', 'Ordinary Dividends', 'Qualified Dividends', 'Business Income']
print ("'Pct Diff' represents the percentage change between the current and new sums")
inc_df

'Pct Diff' represents the percentage change between the current and new sums


,New,Current,SOI,Pct Diff,New - SOI,Current - SOI
WAS,"6,784,950,833,499.0","6,784,952,125,431.1","6,785,880,966,000.0",-0.0,"-930,132,501.0","-928,840,568.9"
Taxable Interest,"90,425,786,010.6","93,894,174,624.3","93,894,281,000.0",-3.7,"-3,468,494,989.4","-106,375.7"
Ordinary Dividends,"254,696,960,387.0","254,702,137,258.3","254,702,232,000.0",-0.0,"-5,271,613.0","-94,741.7"
Qualified Dividends,"180,830,300,363.9","186,804,634,087.7","192,447,708,000.0",-3.2,"-11,617,407,636.1","-5,643,073,912.3"
Business Income,"317,260,010,213.3","317,258,657,428.4","317,248,783,000.0",0.0,"11,227,213.3","9,874,428.4"


# Distribution of Income Variables

In [16]:
new_dist_dict = {}
cur_dist_dict = {}
for item in inc_vars:
    new_dist_dict[item] = distribution(calc_new.array(item), calc_new.array('s006'), calc_new.array('c00100'))
    cur_dist_dict[item] = distribution(calc.array(item), calc.array('s006'), calc.array('c00100'))
new_dist = pd.DataFrame()
cur_dist = pd.DataFrame()
# Add all of the variables to a data frame
for tc, x in zip(inc_vars, soi_inc_vars):
    new_dist[x] = new_dist_dict[tc][1]
    cur_dist[x] = cur_dist_dict[tc][1]
new_dist['AGI Bin'] = index_list()
new_dist['label'] = 'New'
cur_dist['AGI Bin'] = index_list()
cur_dist['label'] = 'Current'
# SOI distribution
items_tups = [('WAS', 'WAS'), ('INT', 'Interest Income'), ('ODIV', 'Ordinary Dividends'),
              ('QDIV', 'Qualified Dividends'), ('BIZ', 'Business Income')]
soi_dist = pd.DataFrame()
for item in items_tups:
    soi_dist[item[0]] = (soi_income[item[0]] / soi_income[item[0]].sum())
soi_dist['AGI Bin'] = index_list()
soi_dist['label'] = 'SOI'

scatter_new = ColumnDataSource(new_dist)
scatter_cur = ColumnDataSource(cur_dist)
scatter_soi = ColumnDataSource(soi_dist)

figure_list = []
for item in items_tups:
    title = 'Percent of Total {} by AGI Bin'.format(item[1])
    f = figure(title=title, x_range=index_list())
    f.xaxis.major_label_orientation = 45
    f.circle(x='AGI Bin', y=item[0], color='green', size=10, legend='New', alpha=0.5,
             source=scatter_new)
    f.circle(x='AGI Bin', y=item[0], color='red', size=10, legend='Current', alpha=0.5,
             source=scatter_cur)
    f.circle(x='AGI Bin', y=item[0], color='blue', size=10, legend='SOI', alpha=0.5,
             source=scatter_soi)
    f.legend.location = 'top_left'
    f.yaxis[0].formatter = NumeralTickFormatter(format='0.00%')
    hover = HoverTool(tooltips=[('Pct', '@{}%'.format(item[0]))])
    f.add_tools(hover)
    figure_list.append(f)
show(column(figure_list))

In [17]:
new_tot = pd.DataFrame()
cur_tot = pd.DataFrame()
# Add all of the variables to a data frame
for tc, x in zip(inc_vars, soi_inc_vars):
    new_tot[x] = new_dist_dict[tc][0]
    cur_tot[x] = cur_dist_dict[tc][0]
new_tot['AGI Bin'] = index_list()
new_tot['label'] = 'New'
cur_tot['AGI Bin'] = index_list()
cur_tot['label'] = 'Current'

soi_income['AGI Bin'] = index_list()
soi_income['label'] = 'SOI'

total_new = ColumnDataSource(new_tot)
total_cur = ColumnDataSource(cur_tot)
total_soi = ColumnDataSource(soi_income)

figure_list = []
for item in items_tups:
    title = 'Total {} by AGI Bin'.format(item[1])
    f = figure(title=title, x_range=index_list())
    f.xaxis.major_label_orientation = 45
    f.circle(x='AGI Bin', y=item[0], color='green', size=10, legend='New', alpha=0.5,
             source=total_new)
    f.circle(x='AGI Bin', y=item[0], color='red', size=10, legend='Current', alpha=0.5,
             source=total_cur)
    f.circle(x='AGI Bin', y=item[0], color='blue', size=10, legend='SOI', alpha=0.5,
             source=total_soi)
    f.legend.location = 'top_left'
    f.yaxis[0].formatter = NumeralTickFormatter(format='$0.00a')
    hover = HoverTool(tooltips=[('Pct', '@{}%'.format(item[0]))])
    f.add_tools(hover)
    figure_list.append(f)
show(column(figure_list))

# Itemized Deduction Amounts

In [18]:
deductions_new = {}
deductions_cur = {}
deduction_list = ['Medical Expenses', 'State and Local Taxes', 'Real Estate Taxes',
                  'Interest Paid', 'Charitable Cash Contributions',
                  'Charitable Non-Cash Contributions', 'Total Misc. Expenses',
                  'Net Casualty or Loss']
deduction_vars = ['e17500', 'e18400', 'e18500', 'e19200', 'e19800', 'e20100',
                  'e20400', 'g20500']
for ded, var in zip(deduction_list, deduction_vars):
    deductions_new[ded] = (new_df[var][new_df['c04470'] > 0] *
                           new_df['s006'][new_df['c04470'] > 0]).sum()
    deductions_cur[ded] = (calc_df[var][calc_df['c04470'] > 0] *
                           calc_df['s006'][calc_df['c04470'] > 0]).sum()
ded_new_df = pd.DataFrame.from_dict(deductions_new, 'index')
ded_new_df.columns = ['Total']
ded_new_df['source'] = 'New'
ded_cur_df = pd.DataFrame.from_dict(deductions_cur, 'index')
ded_cur_df.columns = ['Total']
ded_cur_df['source'] = 'Current'

soi_deductions['source'] = 'SOI'

ded_cds_new = ColumnDataSource(ded_new_df)
ded_cds_new.add(data=[i - .5 for i in range(1, 9)], name='xaxis')
ded_cds_cur = ColumnDataSource(ded_cur_df)
ded_cds_cur.add(data=[i - .25 for i in range(1, 9)], name='xaxis')
ded_cds_soi = ColumnDataSource(soi_deductions)
ded_cds_soi.add(data=[i - .75 for i in range(1, 9)], name='xaxis')

'xaxis'

In [19]:
xaxis_list = ['Interest Paid', 'Total Misc. Expenses', 'Medical Expenses',
              'Charitable Non-Cash Contributions', 'Real Estate Taxes',
              'State and Local Taxes', 'Charitable Cash Contributions',
              'Net Casualty or Loss']
f = figure(title='Itemized Deduction Totals', x_range=xaxis_list, width=800)
#f = figure(title='Itemized Deduction Totals')
f.xaxis.major_label_orientation = 45
f.vbar(x='xaxis', bottom=0, top='Total', legend='New',
       color='green', alpha=0.5, line_alpha=1,
       width=0.25, source=ded_cds_new)
f.vbar(x='xaxis', bottom=0, top='Total', legend='Current', color='red',
       alpha=0.5, line_alpha=1,
       width=0.25, source=ded_cds_cur)
f.vbar(x='xaxis', bottom=0, top='Total', legend='SOI', color='blue',
       alpha=0.5, line_alpha=1,
       width=0.25, source=ded_cds_soi)
hover = HoverTool(tooltips=[('Total', '@Total{$0.00a}')])
f.yaxis[0].formatter = NumeralTickFormatter(format='$0.00a')
f.add_tools(hover)

show(f)

In [20]:
ded_error_df = pd.DataFrame()
ded_error_df['Difference: New'] = ded_new_df['Total'] - soi_deductions['Total']
ded_error_df['% Difference: New'] = 100 * ded_error_df['Difference: New'] / soi_deductions['Total']
ded_error_df['Difference: Current'] = ded_cur_df['Total'] - soi_deductions['Total']
ded_error_df['% Difference: Current'] = 100 * ded_error_df['Difference: Current'] / soi_deductions['Total']
print ('Error in Itemized Deductions Relative to SOI Totals')
ded_error_df

Error in Itemized Deductions Relative to SOI Totals


,Difference: New,% Difference: New,Difference: Current,% Difference: Current
Medical Expenses,"-10,881,857,435.6",-8.4,"-10,263,322,756.7",-8.0
State and Local Taxes,"-17,335,117,915.7",-5.3,"-15,964,512,279.0",-4.9
Real Estate Taxes,"3,655,391,628.2",2.0,"17,294,916,696.2",9.6
Interest Paid,"-14,394,297,495.7",-4.7,"-7,293,196,129.2",-2.4
Charitable Cash Contributions,"-5,758,509,001.9",-3.7,"-4,690,078,031.9",-3.0
Charitable Non-Cash Contributions,"-27,044,511,451.0",-41.4,"-31,545,208,332.2",-48.3
Total Misc. Expenses,"4,062,221,013.8",3.2,"4,144,485,540.8",3.3
Net Casualty or Loss,"2,863,801,247.1",129.9,"2,271,378,406.5",103.0


### Itemizing Rate

In [21]:
calc_df['itemizer'] = np.where(calc_df['c04470'] > 0, 1, 0)
new_df['itemizer'] = np.where(new_df['c04470'] > 0, 1, 0)
pr_new = percentile(new_df, 'itemizer', 100, 'e00200', 's006')
pr_cur = percentile(calc_df, 'itemizer', 100, 'e00200', 's006')
item_index = pr_new.index
pr_cds_new = ColumnDataSource({'rate': pr_new,
                               'index': item_index})
pr_cds_cur = ColumnDataSource({'rate': pr_cur,
                                'index': item_index})

hover_ded = HoverTool(tooltips=[('Percentile', '@index'), ('Itemizing Rate', '@rate{0.00%}')])
f = figure(title='Itemizing Rate by Wage Percentile',
           x_axis_label='Wage Percentile')
f.add_tools(hover_ded)
f.line('index', 'rate', legend='New', source=pr_cds_new)
f.line('index', 'rate', legend='Current', color='red', source=pr_cds_cur)
f.yaxis[0].formatter = NumeralTickFormatter(format='0.00%')
f.legend.location = 'top_left'
show(f)

In [22]:
itemizers_new = copy.deepcopy(new_df[new_df['c04470'] > 0])
mean_new = percentile(itemizers_new, 'c04470', 100, 'e00200', 's006')
itemizers = copy.deepcopy(calc_df[calc_df['c04470'] > 0])
mean_cur = percentile(itemizers, 'c04470', 100, 'e00200', 's006')
mean_cds_new = ColumnDataSource({'mean': mean_new,
                                 'xaxis': item_index})
mean_cds_cur = ColumnDataSource({'mean': mean_cur,
                                 'xaxis': item_index})

hover = HoverTool(tooltips=[('Percentile', '@xaxis'), ('Mean', '@mean{$0.00a}')])
f = figure(title='Mean Itemized Deduction Total Among Itemizers',
           x_axis_label='Wage Percentile')
f.add_tools(hover)
f.line('xaxis', 'mean', legend='New', source=mean_cds_new)
f.line('xaxis', 'mean', legend='Current', color='red', source=mean_cds_cur)
f.legend.location = 'top_left'
f.yaxis[0].formatter = NumeralTickFormatter(format='$0.00a')
show(f)

# Tax Liability by Year

In [23]:
# new liabilities
li_new = {'Income': [calc_new.weighted_total('iitax')],
          'Payroll': [calc_new.weighted_total('payrolltax')],
          'Combined': [calc_new.weighted_total('combined')],
          'source': ['New'],
          'Units': [calc_new.array('s006').sum()]}
# current liabilities
li_cur = {'Income': [calc.weighted_total('iitax')],
          'Payroll': [calc.weighted_total('payrolltax')],
          'Combined': [calc.weighted_total('combined')],
          'source': ['Current'],
          'Units': [calc.array('s006').sum()]}
years = [2014]
for year in range(2015, 2028):
    years.append(year)
    calc.advance_to_year(year)
    calc.calc_all()
    calc_new.advance_to_year(year)
    print('new:', calc_new.policy_current_year(), 'current:', calc.policy_current_year())
    print(calc_new.param('_II_em'))
    print(calc.param('_II_em'))
    calc_new.calc_all()
    li_new['Income'].append(calc_new.weighted_total('iitax'))
    li_new['Payroll'].append(calc_new.weighted_total('payrolltax'))
    li_new['Combined'].append(calc_new.weighted_total('combined'))
    li_new['source'].append('New')
    li_new['Units'].append(calc_new.array('s006').sum())
    li_cur['Income'].append(calc.weighted_total('iitax'))
    li_cur['Payroll'].append(calc.weighted_total('payrolltax'))
    li_cur['Combined'].append(calc.weighted_total('combined'))
    li_cur['source'].append('Current')
    li_cur['Units'].append(calc.array('s006').sum())
li_new['years'] = years
li_cur['years'] = years

new: 2015 current: 2015
[ 3900.   3950.   4000.   4050.   4050.      0.      0.      0.      0.
     0.      0.      0.      0.   4883.   4987.5]
[ 3900.    3950.    4000.    4050.    4050.       0.       0.       0.
     0.       0.       0.       0.       0.    4883.    4987.98]
new: 2016 current: 2016
[ 3900.   3950.   4000.   4050.   4050.      0.      0.      0.      0.
     0.      0.      0.      0.   4883.   4987.5]
[ 3900.    3950.    4000.    4050.    4050.       0.       0.       0.
     0.       0.       0.       0.       0.    4883.    4987.98]
new: 2017 current: 2017
[ 3900.   3950.   4000.   4050.   4050.      0.      0.      0.      0.
     0.      0.      0.      0.   4883.   4987.5]
[ 3900.    3950.    4000.    4050.    4050.       0.       0.       0.
     0.       0.       0.       0.       0.    4883.    4987.98]
new: 2018 current: 2018
[ 3900.   3950.   4000.   4050.   4050.      0.      0.      0.      0.
     0.      0.      0.      0.   4883.   4987.5]
[ 3900. 

In [24]:
# ColumnDataSources for plotting
li_cds_new = ColumnDataSource(li_new)
li_cds_cur = ColumnDataSource(li_cur)

In [25]:
f = figure(title='Tax Liabilities 2014-2027')
f.width = 800
f.height = 500
f.line('years', 'Income', color='blue', line_width=2, legend='Income-Current',
       source=li_cds_cur)
f.line('years', 'Income', color='cyan', line_width=2, legend='Income-New',
       source=li_cds_new)
f.line('years', 'Payroll', color='purple', line_width=2, legend='Payroll-Current',
       source=li_cds_cur)
f.line('years', 'Payroll', color='magenta', line_width=2, legend='Payroll-New',
       source=li_cds_new)
f.line('years', 'Combined', color='green', line_width=2, legend='Combined-Current',
       source=li_cds_cur)
f.line('years', 'Combined', color='lime', line_width=2, legend='Combined-New',
       source=li_cds_new)

c1 = f.circle('years', 'Income', color='blue', size=5, legend='Income-Current',
              source=li_cds_cur)
c2 = f.circle('years', 'Income', color='cyan', size=5, legend='Income-New',
              source=li_cds_new)
c3 = f.circle('years', 'Payroll', color='purple', size=5, legend='Payroll-Current',
              source=li_cds_cur)
c4 = f.circle('years', 'Payroll', color='magenta', size=5, legend='Payroll-New',
              source=li_cds_new)
c5 = f.circle('years', 'Combined', color='green', size=5, legend='Combined-Current',
              source=li_cds_cur)
c6 = f.circle('years', 'Combined', color='lime', size=5, legend='Combined-New',
              source=li_cds_new)
f.legend.location = 'top_left'
f.xaxis[0].ticker.desired_num_ticks = 12
f.xaxis.minor_tick_line_color = None
f.yaxis[0].formatter = NumeralTickFormatter(format='$0.00a')

hover = HoverTool(tooltips=[('PUF', '@source'), ('Year', '@years'),
                            ('Income', '@Income{$0.00a}')],
                  renderers=[c1, c2])
hover2 = HoverTool(tooltips=[('PUF', '@source'), ('Year', '@years'),
                             ('Payroll', '@Payroll{$0.00a}')],
                  renderers=[c3, c4])
hover3 =HoverTool(tooltips=[('PUF', '@source'), ('Year', '@years'),
                            ('Combined', '@Combined{$0.00a}')],
                  renderers=[c5, c6])
f.add_tools(hover, hover2, hover3)
show(f)

In [26]:
ldf_current = pd.DataFrame(li_cur, index=years)
ldf_current.drop(['source', 'years'], axis=1, inplace=True)
print ('Current Tax Liabilities by Year')
ldf_current

Current Tax Liabilities by Year


,Combined,Income,Payroll,Units
2014,"2,324,811,726,624.2","1,371,011,564,057.3","953,800,162,566.9","162,224,619.6"
2015,"2,496,405,141,206.3","1,499,687,317,297.4","996,717,823,908.9","164,426,467.9"
2016,"2,572,374,238,835.4","1,539,019,632,733.9","1,033,354,606,101.5","167,110,179.0"
2017,"2,725,712,039,147.9","1,637,704,761,092.9","1,088,007,278,055.0","170,054,950.4"
2018,"2,667,916,509,164.4","1,530,852,268,398.2","1,137,064,240,766.3","172,859,803.9"
2019,"2,773,772,497,820.2","1,591,742,617,388.6","1,182,029,880,431.7","175,361,959.7"
2020,"2,878,645,055,893.6","1,654,554,132,252.4","1,224,090,923,641.2","177,650,713.2"
2021,"2,994,094,216,895.6","1,725,484,401,397.6","1,268,609,815,498.0","179,925,517.3"
2022,"3,122,359,171,982.6","1,805,000,695,989.7","1,317,358,475,993.0","182,192,965.7"
2023,"3,262,271,239,700.0","1,892,909,990,596.5","1,369,361,249,103.5","184,426,496.8"


In [27]:
ldf_new = pd.DataFrame(li_new, index=years)
ldf_new.drop(['source', 'years'], axis=1, inplace=True)
print ('New Tax Liabilities by Year')
ldf_new

New Tax Liabilities by Year


,Combined,Income,Payroll,Units
2014,"2,342,949,614,024.1","1,385,944,534,350.6","957,005,079,673.5","162,065,569.3"
2015,"2,515,439,715,956.5","1,514,641,928,946.7","1,000,797,787,009.8","164,306,179.3"
2016,"2,596,092,562,666.2","1,557,911,726,267.3","1,038,180,836,398.9","167,034,390.5"
2017,"2,750,057,828,074.0","1,657,844,416,173.6","1,092,213,411,900.4","170,025,489.8"
2018,"2,694,753,742,864.4","1,553,716,193,714.9","1,141,037,549,149.5","172,840,481.4"
2019,"2,803,341,846,513.7","1,617,521,705,791.1","1,185,820,140,722.6","175,383,045.8"
2020,"2,909,754,274,835.8","1,681,957,562,159.6","1,227,796,712,676.3","177,669,696.8"
2021,"3,026,704,779,226.1","1,754,246,287,884.2","1,272,458,491,341.9","179,962,345.5"
2022,"3,156,242,823,655.6","1,834,923,534,723.0","1,321,319,288,932.6","182,213,231.7"
2023,"3,298,051,083,167.8","1,924,655,796,812.9","1,373,395,286,355.0","184,481,246.9"


In [28]:
print ('Difference in Tax Liabilities by Year')
diff_df = ldf_new - ldf_current
diff_df = diff_df.append(diff_df.sum(), ignore_index=True)
years.append('Total')
diff_df.index = years
diff_df

Difference in Tax Liabilities by Year


,Combined,Income,Payroll,Units
2014,"18,137,887,399.9","14,932,970,293.2","3,204,917,106.7","-159,050.3"
2015,"19,034,574,750.2","14,954,611,649.3","4,079,963,100.9","-120,288.6"
2016,"23,718,323,830.8","18,892,093,533.4","4,826,230,297.4","-75,788.5"
2017,"24,345,788,926.0","20,139,655,080.7","4,206,133,845.3","-29,460.6"
2018,"26,837,233,700.0","22,863,925,316.7","3,973,308,383.2","-19,322.5"
2019,"29,569,348,693.4","25,779,088,402.5","3,790,260,290.9","21,086.1"
2020,"31,109,218,942.2","27,403,429,907.2","3,705,789,035.1","18,983.6"
2021,"32,610,562,330.5","28,761,886,486.6","3,848,675,843.9","36,828.2"
2022,"33,883,651,673.0","29,922,838,733.4","3,960,812,939.6","20,265.9"
2023,"35,779,843,467.8","31,745,806,216.4","4,034,037,251.4","54,750.1"


In [29]:
print ('Pct. Change in Tax Liabilities by Year')
((ldf_new - ldf_current) / ldf_current) * 100

Pct. Change in Tax Liabilities by Year


,Combined,Income,Payroll,Units
2014,0.8,1.1,0.3,-0.1
2015,0.8,1.0,0.4,-0.1
2016,0.9,1.2,0.5,-0.0
2017,0.9,1.2,0.4,-0.0
2018,1.0,1.5,0.3,-0.0
2019,1.1,1.6,0.3,0.0
2020,1.1,1.7,0.3,0.0
2021,1.1,1.7,0.3,0.0
2022,1.1,1.7,0.3,0.0
2023,1.1,1.7,0.3,0.0


In [30]:
show_df = pd.DataFrame(OrderedDict({'Year': [i for i in range(2014, 2028)],
                        'Units - New': [round(i / 1e6, 1) for i in li_new['Units']],
                        'Units - Old': [round(i / 1e6, 1) for i in li_cur['Units']],
                        'PTax - New': [round(i / 1e9, 1) for i in li_new['Payroll']],
                        'PTax - Old': [round(i / 1e9, 1) for i in li_cur['Payroll']],
                        'ITax - New': [round(i / 1e9, 1) for i in li_new['Income']],
                        'ITax - Old': [round(i / 1e9, 1) for i in li_cur['Income']]}))
print ('Units in millions, PTax and ITax in billions')
show_df

Units in millions, PTax and ITax in billions


,Year,Units - New,Units - Old,PTax - New,PTax - Old,ITax - New,ITax - Old
0,2014,162.1,162.2,957.0,953.8,"1,385.9","1,371.0"
1,2015,164.3,164.4,"1,000.8",996.7,"1,514.6","1,499.7"
2,2016,167.0,167.1,"1,038.2","1,033.4","1,557.9","1,539.0"
3,2017,170.0,170.1,"1,092.2","1,088.0","1,657.8","1,637.7"
4,2018,172.8,172.9,"1,141.0","1,137.1","1,553.7","1,530.9"
5,2019,175.4,175.4,"1,185.8","1,182.0","1,617.5","1,591.7"
6,2020,177.7,177.7,"1,227.8","1,224.1","1,682.0","1,654.6"
7,2021,180.0,179.9,"1,272.5","1,268.6","1,754.2","1,725.5"
8,2022,182.2,182.2,"1,321.3","1,317.4","1,834.9","1,805.0"
9,2023,184.5,184.4,"1,373.4","1,369.4","1,924.7","1,892.9"
